In [4]:
import pandas as pd 
import random
from tqdm import tqdm

N_COLS = 640
N_ROWS = 480

# n positives and n negatives for each scale
PATCHES_EACH_ROW = 16
SCALES = [32,64]

DATA = "spqr_autolabel_and_manual_train.csv"

df = pd.read_csv(DATA)


,id,image,frame_contains_ball,center_x,center_y,radius
0,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.084630
1,1,08743_RoboCup2019_HULKs_Daredevil-1st_16903389...,False,NaN,NaN,NaN
2,2,08455_RoboCup2019_HULKs_Daredevil-1st_16423278...,False,NaN,NaN,NaN
3,3,09587_RoboCup2019_HULKs_Frogthor-1st_16633141_...,True,126.187200,284.892357,30.527795
4,4,08896_RoboCup2019_HULKs_Daredevil-2nd_17678880...,False,NaN,NaN,NaN
...,...,...,...,...,...,...
8615,8615,00150_RoboCup2019_HTWK-Leipzig_Venom-2nd_20665...,True,208.280707,266.561391,26.228377
8616,8616,00631_RoboCup2019_HTWK-Leipzig_Otto-1st_869429...,True,261.909957,313.774505,28.423629
8617,8617,00636_RoboCup2019_HTWK-Leipzig_Otto-2nd_156989...,True,287.522038,361.031675,33.556893
8618,8618,00647_RoboCup2019_HTWK-Leipzig_Otto-1st_869429...,True,228.193792,401.766569,52.281234


In [5]:
import pandas as pd
import random
from tqdm import tqdm

def is_ball_within_patch(row, patch_coordinates, patch_dim):
    return row["frame_contains_ball"] and \
        patch_coordinates[0] < row["center_x"] < patch_coordinates[0] + patch_dim and \
        patch_coordinates[1] < row["center_y"] < patch_coordinates[1] + patch_dim 

patches_list = []

for _, sample in tqdm(df.iterrows()):

    for scale in SCALES:

        for _ in range(PATCHES_EACH_ROW):

            # GENERATE NEGATIVE
            while True:
                r_x = random.randint(0, N_COLS - scale - 1)
                r_y = random.randint(0, N_ROWS - scale - 1)

                if not is_ball_within_patch(sample, (r_x, r_y), scale):
                    row = sample.to_dict()
                    row["patch_x"] = r_x
                    row["patch_y"] = r_y
                    row["patch_size"] = scale
                    row["patch_contains_ball"] = False
                    patches_list.append(row)
                    break

            # GENERATE POSITIVE
            while sample["frame_contains_ball"]:
                min_x = max(int(sample["center_x"]) - scale + 1, 0)
                min_y = max(int(sample["center_y"]) - scale + 1, 0)

                if int(sample["center_x"]) <= 1 or int(sample["center_y"]) <=1:
                    break

                r_x = random.randint(min_x, int(sample["center_x"]) - 1)
                r_y = random.randint(min_y, int(sample["center_y"]) - 1)

                if is_ball_within_patch(sample, (r_x, r_y), scale):
                    row = sample.to_dict()
                    row["patch_x"] = r_x
                    row["patch_y"] = r_y
                    row["patch_size"] = scale
                    row["patch_contains_ball"] = True
                    patches_list.append(row)
                    break

patches_df = pd.DataFrame(patches_list)
patches_df.to_csv(DATA.split(".")[0]+"_patchified.csv", index=False)


8620it [00:09, 941.76it/s] 


In [6]:
patches_df = pd.read_csv(DATA.split(".")[0]+"_patchified.csv")
patches_df

,id,image,frame_contains_ball,center_x,center_y,radius,patch_x,patch_y,patch_size,patch_contains_ball
0,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.08463,343,430,32,False
1,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.08463,434,399,32,True
2,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.08463,519,137,32,False
3,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.08463,417,373,32,True
4,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.08463,331,238,32,False
...,...,...,...,...,...,...,...,...,...,...
417691,8619,00983_RoboCup2019_HTWK-Leipzig_Venom-2nd_22397...,False,NaN,NaN,NaN,144,343,64,False
417692,8619,00983_RoboCup2019_HTWK-Leipzig_Venom-2nd_22397...,False,NaN,NaN,NaN,462,127,64,False
417693,8619,00983_RoboCup2019_HTWK-Leipzig_Venom-2nd_22397...,False,NaN,NaN,NaN,431,415,64,False
417694,8619,00983_RoboCup2019_HTWK-Leipzig_Venom-2nd_22397...,False,NaN,NaN,NaN,427,118,64,False
